In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image

In [2]:
def OrderofYearMonth(file):
    return(file[-13:-4])

#   Y   Y   Y   Y  M  M  _  X  X
# -13 -12 -11 -10 -9 -8 -7 -6 -5

In [3]:
def statistics_boats(Location, Year, jpg_folder, csv_download_folder):
    labels_folder = jpg_folder+"labels/"
    labels_folder_list = os.listdir(labels_folder)
    
    csv_file_name = csv_download_folder + Location + '_' + str(Year) + '.csv'
    
    list_of_unsorted_files=[]
    list_of_files=[]
    small = [0,0,0,0,0,0,0,0,0,0,0,0,0]
    large = [0,0,0,0,0,0,0,0,0,0,0,0,0]
    small_white_boat_in_this_area = [0,0,0,0,0,0,0,0,0,0,0,0,0]
    Months = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'July', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    zero_to_nine = [0,1,2,3,4,5,6,7,8,9]

    # Creates pandas DataFrame with empty data of lists
    data = {}
    Table = pd.DataFrame(data, index =['Boats length < 24 meters','Boats length >= 24 meters', 'Small White Boats'])

    
    # Sorted files
    for file in labels_folder_list:
        if file.endswith('txt'): # check the extension of a file
            list_of_unsorted_files.append(file)
    list_of_files = sorted(list_of_unsorted_files, key = OrderofYearMonth)
    #print(list_of_files)

    
    # Statistics
    for sorted_file in list_of_files:
        for i in range(1,10):
            for j in range(0,2):
                if sorted_file[-9] == str(j) and sorted_file[-8] == str(i):
                    df = pd.read_table(labels_folder+sorted_file, delimiter = " ", names=("Boat=0", "X", "Y", "Width", "Height", "Length"))
                    df['x_min'] = round((df["X"] - 0.5*df["Width"])*3840)
                    df['x_max'] = round((df["X"] + 0.5*df["Width"])*3840)
                    df['y_min'] = round((df["Y"] - 0.5*df["Height"])*2160)
                    df['y_max'] = round((df["Y"] + 0.5*df["Height"])*2160)
                    #print(len(df))
                    for length in df['Length']:
                        if length < 24:
                            small[i+10*j] += 1
                        elif length >= 24:
                            large[i+10*j] += 1
                    
                    ### SMALL WHITE BOATS ###
                    #print(sorted_file)
                    im = Image.open(jpg_folder + sorted_file.replace('txt','jpg'))
                    array = np.array(im)
                    count_white = 0
                    count_all = 0

                    for index in range(0,len(df)): # 1st row, ...
                        #print(x_min)
                        if df['Length'][index] < 24:
                            x_min = int(df['x_min'][index])
                            x_max = int(df['x_max'][index])
                            y_min = int(df['y_min'][index])
                            y_max = int(df['y_max'][index])
                            #print(y_max)
                            for m in range(x_min, x_max):
                                for n in range(y_min,y_max):
                                    s = array[n,m]
                                    g = s[0]*0.299 + s[1]*0.587 + s[2]*0.114 # 根据RGB值判断颜色是否深浅色
                                    if g>80:
                                        count_white += 1
                                        count_all += 1
                                    else:
                                        count_all += 1
                                        pass
                            if ((count_white/count_all)>0.5) is True:
                                small_white_boat_in_this_area[i+10*j] += 1 #/ len(list_of_files)
                            else:
                                pass
                        #print(small_white_boat_in_this_area)
                    ### SMALL WHITE BOATS ###
                    Table[Months[i+10*j]] = [int(small[i+10*j]), int(large[i+10*j]), int(small_white_boat_in_this_area[i+10*j])]
                    

    # print the data
    s = Table.style.set_table_attributes("style='display:inline'").set_caption(Location + " " + str(Year))
    Table.to_csv(csv_file_name)
    return(s)

In [4]:
"""
statistics_boats(Location="", Year=, 
                 labels_folder = "", 
                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")
"""

'\nstatistics_boats(Location="", Year=, \n                 labels_folder = "", \n                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")\n'

In [5]:
# Test block
#statistics_boats(Location="SantaRosalia", Year=2019, 
#                 jpg_folder = "/Users/realgjl/Downloads/", 
#                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")

In [6]:
statistics_boats(Location="SantaRosalia", Year=2019, 
                 jpg_folder = "/Users/realgjl/Desktop/BENV0096/3a_Sharpened_SantaRosalia/2019/", 
                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")

,Jan,July,Aug,Sep,Dec
Boats length < 24 meters,27,41,36,29,22
Boats length >= 24 meters,0,2,1,0,2
Small White Boats,25,41,35,29,22


In [7]:
statistics_boats(Location="SantaRosalia", Year=2020, 
                 jpg_folder = "/Users/realgjl/Desktop/BENV0096/3a_Sharpened_SantaRosalia/2020/", 
                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")


,Jan,Feb,Mar,Apr,May,Jun,July,Aug,Sep,Nov,Dec
Boats length < 24 meters,50,43,37,48,8,44,66,58,58,43,57
Boats length >= 24 meters,1,1,1,1,0,1,3,1,1,1,1
Small White Boats,37,42,37,43,6,42,65,54,58,41,56


In [8]:
statistics_boats(Location="SantaRosalia", Year=2021, 
                 jpg_folder = "/Users/realgjl/Desktop/BENV0096/3a_Sharpened_SantaRosalia/2021/", 
                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")

,Apr,May
Boats length < 24 meters,32,46
Boats length >= 24 meters,1,2
Small White Boats,32,46


In [9]:
statistics_boats(Location="Loreto", Year=2019, 
                 jpg_folder = "/Users/realgjl/Desktop/BENV0096/3b_Sharpened_Loreto/2019/", 
                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")

,Jan,Apr,Jun,July,Aug,Sep,Nov,Dec
Boats length < 24 meters,40,60,18,49,33,42,38,17
Boats length >= 24 meters,0,1,0,0,1,0,1,0
Small White Boats,40,55,18,49,33,41,37,16


In [10]:
statistics_boats(Location="Loreto", Year=2020, 
                 jpg_folder = "/Users/realgjl/Desktop/BENV0096/3b_Sharpened_Loreto/2020/", 
                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")

,Jan,Feb,Mar,Apr,Jun,July,Aug,Sep,Nov
Boats length < 24 meters,18,26,18,30,32,33,20,43,43
Boats length >= 24 meters,0,0,0,0,0,0,0,1,1
Small White Boats,18,26,17,28,30,32,20,43,39


In [11]:
statistics_boats(Location="Loreto", Year=2021, 
                 jpg_folder = "/Users/realgjl/Desktop/BENV0096/3b_Sharpened_Loreto/2021/", 
                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")

,Jan,Feb,Apr
Boats length < 24 meters,16,33,33
Boats length >= 24 meters,0,0,0
Small White Boats,16,33,33


In [12]:
statistics_boats(Location="Guaymas", Year=2019, 
                 jpg_folder = "/Users/realgjl/Desktop/BENV0096/3c_Sharpened_Guaymas/2019/", 
                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")

,Jan,Feb,Mar,Apr,Jun,July,Sep,Nov
Boats length < 24 meters,115,83,99,131,82,107,90,98
Boats length >= 24 meters,31,24,21,24,22,22,21,19
Small White Boats,112,83,99,131,79,105,89,96


In [13]:
statistics_boats(Location="Guaymas", Year=2020, 
                 jpg_folder = "/Users/realgjl/Desktop/BENV0096/3c_Sharpened_Guaymas/2020/", 
                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")

,Jan,Feb,Mar,Apr,May,Jun,July,Aug,Sep,Nov,Dec
Boats length < 24 meters,63,101,159,52,145,85,66,121,28,74,113
Boats length >= 24 meters,18,13,15,13,21,4,13,22,15,28,15
Small White Boats,63,101,138,52,142,79,66,119,25,66,108


In [14]:
statistics_boats(Location="Guaymas", Year=2021, 
                 jpg_folder = "/Users/realgjl/Desktop/BENV0096/3c_Sharpened_Guaymas/2021/", 
                 csv_download_folder = "/Users/realgjl/Desktop/BENV0096/5a_CSVs/")

,Feb,Mar,Apr,May
Boats length < 24 meters,161,167,144,119
Boats length >= 24 meters,19,33,21,22
Small White Boats,147,158,144,106
